In [2]:
%run 'import_dbase.ipynb'
%run 'import_rct007.ipynb'
%run 'import_getters.ipynb'
eventTuples = getEvents('rct007').index

/home/ihs01/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [390]:
import pandas as pd
import numpy as np
import scipy as sp
%matplotlib inline
import matplotlib.pyplot as plt   
import matplotlib.dates as dates
import seaborn as sns
from ipywidgets import interact, fixed, widgets
sns.set_context(rc={'lines.markeredgewidth': 0.1})
sns.set_style("dark")
sns.set_style("darkgrid")

%connect_info

{
  "stdin_port": 45354, 
  "ip": "127.0.0.1", 
  "control_port": 58943, 
  "hb_port": 60096, 
  "signature_scheme": "hmac-sha256", 
  "key": "000a8c59-60ad-4059-bdec-6a08e1428d89", 
  "shell_port": 49455, 
  "transport": "tcp", 
  "iopub_port": 46760
}

Paste the above JSON into a file, and connect with:
    $> ipython <app> --existing <file>
or, if you are local, you can connect with just:
    $> ipython <app> --existing /run/user/1000/jupyter/kernel-6b37b387-22f5-4441-99a5-313961232421.json 
or even just:
    $> ipython <app> --existing 
if this is the most recent IPython session you have started.


In [391]:
pd.set_option('display.max_rows', 150)

In [392]:
def slidePlot(dfIn, signal=('Cell (V)',), resample_rate_in_sec=120, start=0, end=1, resampMethod='rms'):
    
    dfIn = dfIn.ix[start:end,:]
    col = list(signal)
    title = signal
    dt = str(resample_rate_in_sec) + 'T'
    if resampMethod == 'rms' : resampMethod = resampleRMS
    
    # get x,y for plot
    out = dfIn[col].resample(dt,how=resampMethod)
    rng = pd.to_datetime(out.index)
    
    fig, ax = plt.subplots(1,1, figsize=[20,12])
    ax.plot_date(rng.to_pydatetime(), out, 'v-', lw=2.5, marker='.')
    fig.autofmt_xdate()

    ax.xaxis.set_major_formatter(dates.DateFormatter('%a %m/%d-%H:%M:%S'))
    ax.xaxis.grid(True)
    ax.yaxis.grid(True)
    
    plt.title(title, fontsize=28, ha="center")  
    plt.tick_params(axis='y', labelsize=18)
    plt.tick_params(which='minor',axis='x', labelsize=18)
    plt.tick_params(which='major', axis='x', labelsize=18)
    plt.legend(col, fontsize=18, loc='best')
    plt.tight_layout()
    fig.canvas.draw()
    return 

def iSlidePlot(df, MAX_RESAMP_TIME=120000*30, DELTA_SAMP=1, DELTA_TIME=1):
    
    %matplotlib inline
    
    attributes = df.columns.tolist()
    MAX_NUM = len(df)-2
    
    sigWid = widgets.SelectMultiple(
    description = "Signal",
    options=df.columns.tolist()[1:])
    
    resampMethodWid = widgets.RadioButtons(
    description='Resample Method',
    options=['mean', 'rms'])
    
    resampRateWid = widgets.IntSlider(
    value=1,
    min=DELTA_SAMP,
    max=MAX_RESAMP_TIME,
    step=DELTA_SAMP,
    description='Resample Rate (ms)', continuous_update=False)

    startWid = widgets.IntSlider(
    value=0,
    min=0,
    max=len(df),
    step=DELTA_TIME,
    description='Start Sample', continuous_update=False)

    endWid = widgets.IntSlider(
    value=len(df),
    min=0,
    max=len(df),
    step=DELTA_TIME,
    description='End Sample', continuous_update=False)
    
    updateWid = widgets.Checkbox(
    description='Update Data',
    value=False)

    interact(slidePlot, dfIn=fixed(df), 
             signal=sigWid, 
             resample_rate_in_sec=resampRateWid, 
             start=startWid, 
             end=endWid,
             resampMethod = resampMethodWid, 
             updateData = updateWid,
             __manual=True)
    return

In [3]:
sensor = 'rct_rtd1'
#First event
start = 1
#Last event
end = 10

In [4]:
df1 = getMultiEvent(sensor, start, end, eventTuples)

TypeError: 'int' object has no attribute '__getitem__'

In [1]:
x = getEngData(sensor, eventTuples[5])

NameError: name 'getEngData' is not defined

In [397]:
df2 = getTempData(eventTuples)

In [398]:
iSlidePlot(df2)

## Events

In [399]:
eventsDf=getEvents('rct007')
#eventsDf

## Input Power

In [400]:
def findInputPower(eventTuple):
    lineVoltageSensor = 'v5'
    fanCurrentSensor = 'v8'
    htr1CurrentSensor = 'v1'
    htr2CurrentSensor = 'v4'
    
    lineVoltage = getEngData(lineVoltageSensor, eventTuple)
    fanCurrent = getEngData(fanCurrentSensor, eventTuple)
    h1Current = getEngData(htr1CurrentSensor, eventTuple)
    h2Current = getEngData(htr2CurrentSensor, eventTuple)

    pwrFan =  lineVoltage.values * fanCurrent.values
    pwrHtr1 = lineVoltage.values * h1Current.values
    pwrHtr2 = lineVoltage.values * h2Current.values
    
    meanPwrFan = pwrFan.mean()
    stdPwrFan = pwrFan.std()
    meanPwrHtr1 = pwrHtr1.mean()
    stdPwrHtr1 = pwrHtr1.std()
    meanPwrHtr2 = pwrHtr2.mean()
    stdPwrHtr2 = pwrHtr2.std()

    return meanPwrFan+meanPwrHtr1+meanPwrHtr2

eventTuples = getEvents('rct007').index
print 'input power: %f'%findInputPower(eventTuples[eventNumber])

input power: 0.882526


## Reactor & Ambient Temperatures

In [401]:
event = 10

In [404]:
df = getEngData('rct_rtd1', eventTuples[event])
for i in range (2, 7):
    df['rct_rtd%d' %i] = getEngData('rct_rtd%d' %i, eventTuples[event])
for i in range (1, 5):
    df['amb_rtd%d' %i] = getEngData('amb_rtd%d' %i, eventTuples[event])


df['rct_mean'] = df[['rct_rtd1','rct_rtd2','rct_rtd3','rct_rtd4','rct_rtd5','rct_rtd6']].mean(axis=1)
df['amb_mean'] = df[['amb_rtd1', 'amb_rtd2', 'amb_rtd3', 'amb_rtd4']].mean(axis=1)
#df['htr_mean'] = df[['htr1_tc', 'htrc2_tc']].mean(axis=1)
#df['amb_var'] = df

In [405]:
df

,rct_rtd1,rct_rtd2,rct_rtd3,rct_rtd4,rct_rtd5,rct_rtd6,amb_rtd1,amb_rtd2,amb_rtd3,amb_rtd4,rct_mean,amb_mean
2016-06-28 22:48:27.377682,75.980,80.429,78.665,78.024,79.758,79.164,22.663,23.216,20.207,19.241,78.670000,21.33175
2016-06-28 22:48:27.869749,75.980,80.398,78.649,78.009,79.773,79.164,22.663,23.231,20.207,19.241,78.662167,21.33550
2016-06-28 22:48:28.326429,75.964,80.429,78.649,78.009,79.773,79.164,22.663,23.231,20.253,19.226,78.664667,21.34325
2016-06-28 22:48:28.769540,76.042,80.429,78.649,78.024,79.758,79.164,22.663,23.216,20.238,19.226,78.677667,21.33575
2016-06-28 22:48:29.211073,76.042,80.414,78.665,78.024,79.742,79.164,22.663,23.216,20.238,19.226,78.675167,21.33575
2016-06-28 22:48:29.644737,75.996,80.414,78.665,78.024,79.742,79.149,22.663,23.216,20.238,19.210,78.665000,21.33175
2016-06-28 22:48:30.078297,75.996,80.414,78.649,78.024,79.726,79.149,22.663,23.262,20.238,19.210,78.659667,21.34325
2016-06-28 22:48:30.520660,75.980,80.429,78.633,78.024,79.726,79.149,22.663,23.262,20.207,19.195,78.656833,21.33175
2016-06-28 22:48:30.953397,76.011,80.429,78.633,78.040,79.758,79.117,22.694,23.216,20.223,19.226,78.664667,21.33975
2016-06-28 22:48:31.404412,76.011,80.414,78.665,78.024,79.742,79.117,22.648,23.216,20.223,19.226,78.662167,21.32825


## Multiple Events

In [406]:
sensor = 'rct_rtd1'
#First event
start = 1
#Last event
end = 10

In [551]:
#The 'put' function allows the getMultiEvent function to fill a column in the dataframe with the Event ID 
#associated with the data
def put(df, column, values):
    df[column] = 0
    np.put(df[column], np.arange(len(df)), values)

def getMultiEvent(sensor, eventTuples, start, end):

    '''
    NAME
        getMultiEvent
    
    DESCRIPTION
        This function gets data from one sensor for multiple, sequential events.
        Requires 'eventTuples' from getEventData
    
    INPUT
        sensor:       name of the sensor to get data for
                      (get this from the list of sensors from getSensorList)
        eventTuples:  event ID, run, database info
        start:        first event #
        end:          last event #
    
    RETURN
        dataframe with data from multiple events
        columns = sensor, event ID
    
    EXAMPLE
        sensor = 'rct_rtd1'
        start = 1
        end = 10
        
        df = getMultiEvent(sensor, eventTuples, start, end)
    '''
    
    df = getEngData(sensor, eventTuples[start])
    put(df, 'Event ID', [start])
    frames = [df]

    for i in range ((start+1), (end+1)):
        df1 = getEngData(sensor, eventTuples[i])
        put(df1, 'Event ID', [i])
        frames.append(df1)
        df = pd.concat(frames)
    
    return df

In [550]:
eventTuples[1:10]

,,,Start,Stop,Duty_Cycle,Gas,Pressure
eventId,dataBase,run,,,,,
1,rct007,004,2016-06-28 02:46:47.820485,2016-06-28 03:16:49.830191,0.05,Vacuum,0.000028
2,rct007,004,2016-06-28 04:46:56.306332,2016-06-28 05:16:58.364696,0.1,Vacuum,0.000028
3,rct007,004,2016-06-28 07:07:06.402626,2016-06-28 07:37:08.680572,0.2,Vacuum,0.000028
4,rct007,004,2016-06-28 09:17:15.895275,2016-06-28 09:47:17.810393,0.4,Vacuum,0.000028
5,rct007,004,2016-06-28 11:57:27.140698,2016-06-28 12:27:29.471676,0.6,Vacuum,0.000028
6,rct007,004,2016-06-28 15:07:41.222185,2016-06-28 15:37:43.563907,0,Vacuum,0.000028
7,rct007,004,2016-06-28 15:50:06.504234,2016-06-28 16:20:08.797504,0,Vacuum,0.000042
8,rct007,004,2016-06-28 17:50:15.421791,2016-06-28 18:20:17.367728,0.05,Vacuum,0.000042
9,rct007,004,2016-06-28 20:28:16.966737,2016-06-28 20:58:19.125531,0.1,Vacuum,0.00004


In [553]:
sensor = 'amb_rtd1'
start = 1
end = 10
df4 = getMultiEvent(sensor, eventTuples, start, end)

KeyError: 1

In [420]:
df4

,amb_rtd1,Event ID
2016-06-28 02:46:47.837618,22.602,1
2016-06-28 02:46:48.301979,22.602,1
2016-06-28 02:46:48.737744,22.648,1
2016-06-28 02:46:49.214098,22.648,1
2016-06-28 02:46:49.662127,22.648,1
2016-06-28 02:46:50.096348,22.602,1
2016-06-28 02:46:50.539857,22.602,1
2016-06-28 02:46:50.972734,22.632,1
2016-06-28 02:46:51.414629,22.617,1
2016-06-28 02:46:51.870699,22.617,1


In [410]:
df2 = getTempData(eventTuples)

In [411]:
df2

,rct_rtd1,rct_rtd2,rct_rtd3,rct_rtd4,rct_rtd5,rct_rtd6,amb_rtd1,amb_rtd2,amb_rtd3,amb_rtd4,rct_mean,amb_mean,flowin_rtd,flowout_rtd,htr1_tc,htr2_tc,htr_mean
2016-06-28 00:46:39.556908,17.247,17.247,17.171,17.263,17.294,17.263,22.632,23.154,18.505,16.542,17.247500,20.20825,15.317,15.730,17.6,17.6,17.60
2016-06-28 00:46:39.989594,17.278,17.217,17.171,17.263,17.263,17.278,22.632,23.170,18.505,16.542,17.245000,20.21225,15.317,15.730,17.6,17.7,17.65
2016-06-28 00:46:40.422867,17.278,17.217,17.155,17.294,17.309,17.278,22.632,23.170,18.520,16.573,17.255167,20.22375,15.332,15.730,17.6,17.7,17.65
2016-06-28 00:46:40.856576,17.294,17.232,17.186,17.294,17.309,17.263,22.648,23.185,18.505,16.542,17.263000,20.22000,15.332,15.730,17.6,17.7,17.65
2016-06-28 00:46:41.289848,17.278,17.232,17.186,17.278,17.309,17.263,22.663,23.185,18.505,16.542,17.257667,20.22375,15.301,15.746,17.6,17.7,17.65
2016-06-28 00:46:41.723590,17.278,17.232,17.171,17.294,17.309,17.263,22.663,23.139,18.505,16.527,17.257833,20.20850,15.317,15.746,17.6,17.7,17.65
2016-06-28 00:46:42.156401,17.278,17.247,17.171,17.294,17.294,17.232,22.648,23.170,18.505,16.527,17.252667,20.21250,15.317,15.715,17.5,17.5,17.50
2016-06-28 00:46:42.631537,17.278,17.247,17.171,17.278,17.278,17.232,22.648,23.170,18.505,16.558,17.247333,20.22025,15.317,15.730,17.5,17.5,17.50
2016-06-28 00:46:43.065383,17.263,17.232,17.171,17.278,17.278,17.232,22.617,23.154,18.520,16.558,17.242333,20.21225,15.317,15.730,17.5,17.5,17.50
2016-06-28 00:46:43.523949,17.263,17.232,17.171,17.309,17.294,17.263,22.648,23.154,18.520,16.558,17.255333,20.22000,15.301,15.715,17.5,17.5,17.50


In [522]:
sensor = 'amb_rtd1'

In [523]:
getStats(sensor, eventTuples[10])

mean		22.6897675648 
max		[ 22.801] 
min		[ 22.586] 
deviation	0.0394312093093 
skew		[ 0.20623031] 
kurtosis	[ 2.29516189]


In [415]:
pd.DataFrame.describe(df2)

,rct_rtd1,rct_rtd2,rct_rtd3,rct_rtd4,rct_rtd5,rct_rtd6,amb_rtd1,amb_rtd2,amb_rtd3,amb_rtd4,rct_mean,amb_mean,flowin_rtd,flowout_rtd,htr1_tc,htr2_tc,htr_mean
count,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000
mean,17.012019,16.978528,16.908507,17.026526,17.027165,17.005470,22.609995,23.155808,18.465066,16.547083,16.993036,20.194488,15.321135,15.782145,17.546427,17.305159,17.425793
std,0.121863,0.121724,0.119141,0.128023,0.124948,0.120711,0.040417,0.030352,0.032807,0.022375,0.121919,0.023437,0.021893,0.031436,0.582677,0.156062,0.312711
min,16.788000,16.772000,16.696000,16.772000,16.803000,16.788000,22.510000,23.062000,18.367000,16.466000,16.792667,20.124000,15.225000,15.669000,16.900000,17.000000,17.000000
25%,16.910000,16.880000,16.803000,16.910000,16.926000,16.910000,22.571000,23.139000,18.443000,16.527000,16.889833,20.177750,15.301000,15.761000,17.200000,17.200000,17.200000
50%,16.987000,16.956000,16.895000,17.002000,17.002000,16.987000,22.602000,23.154000,18.459000,16.542000,16.971667,20.193250,15.317000,15.776000,17.300000,17.300000,17.350000
75%,17.110000,17.079000,17.002000,17.125000,17.125000,17.094000,22.648000,23.185000,18.489000,16.558000,17.094292,20.212250,15.332000,15.807000,17.800000,17.400000,17.550000
max,17.294000,17.247000,17.186000,17.340000,17.309000,17.294000,22.724000,23.262000,18.566000,16.619000,17.263000,20.266000,15.409000,15.868000,19.200000,17.800000,18.500000


In [416]:
pd.DataFrame.kurtosis(df2)

rct_rtd1      -0.974670
rct_rtd2      -0.992780
rct_rtd3      -0.992527
rct_rtd4      -0.936876
rct_rtd5      -0.971291
rct_rtd6      -0.972703
amb_rtd1      -0.672918
amb_rtd2      -0.012330
amb_rtd3      -0.293217
amb_rtd4       0.070383
rct_mean      -1.000635
amb_mean      -0.155215
flowin_rtd     0.796255
flowout_rtd    0.219886
htr1_tc        0.634714
htr2_tc       -0.319360
htr_mean       0.581838
dtype: float64

In [417]:
pd.DataFrame.skew(df2)

rct_rtd1       0.385953
rct_rtd2       0.390673
rct_rtd3       0.386630
rct_rtd4       0.394173
rct_rtd5       0.394944
rct_rtd6       0.396120
amb_rtd1       0.063613
amb_rtd2      -0.030721
amb_rtd3       0.052099
amb_rtd4       0.004551
rct_mean       0.398807
amb_mean      -0.134251
flowin_rtd     0.018491
flowout_rtd   -0.295568
htr1_tc        1.390053
htr2_tc        0.440249
htr_mean       1.165222
dtype: float64

In [432]:
df5 = pd.DataFrame.describe(df4)
df5.drop('Event ID', axis=1, inplace=True)

In [433]:
df5

,amb_rtd1
count,39081.000000
mean,22.710452
std,0.052405
min,22.525000
25%,22.678000
50%,22.709000
75%,22.740000
max,22.909000


In [435]:
df6 = pd.DataFrame.kurtosis(df4)

In [446]:
type(df6)

pandas.core.series.Series

In [484]:
df7 = pd.Series.to_frame(df6)
df7

,0
amb_rtd1,-0.065557
Event ID,-1.217890


In [471]:
#df8 = pd.DataFrame.transpose(df7)
#df8.drop(df8.columns[1], axis=1, inplace=True)

In [489]:
df5.loc[len(df5)] = df7.ix[:1]

In [490]:
df5

,amb_rtd1
count,39081.000000
mean,22.710452
std,0.052405
min,22.525000
25%,22.678000
50%,22.709000
75%,22.740000
max,22.909000
8,-0.065557


In [491]:
df8 = pd.DataFrame.skew(df4)
df9 = pd.Series.to_frame(df8)
df5.loc[len(df5)] = df9.ix[:1]

In [492]:
df5

,amb_rtd1
count,39081.000000
mean,22.710452
std,0.052405
min,22.525000
25%,22.678000
50%,22.709000
75%,22.740000
max,22.909000
8,-0.065557
9,0.041118


In [509]:
df10 = pd.Series(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'kurtosis', 'skew'])

In [519]:
df5.set_index(df10)

,amb_rtd1
count,39081.000000
mean,22.710452
std,0.052405
min,22.525000
25%,22.678000
50%,22.709000
75%,22.740000
max,22.909000
kurtosis,-0.065557
skew,0.041118


In [541]:
def getDFStats (sensor, eventTuples):
    
    data = getEngData(sensor, eventTuples)
    
    df = pd.DataFrame.describe(data)
    
    array1 = sp.stats.kurtosis(data, fisher=False)
    series1 = pd.DataFrame.skew(data)
    series2 = pd.Series(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'kurtosis', 'skew'])
    
    df1 = pd.Series.to_frame(series1)
    
    df.loc[len(df)] = array1[0]
    df.loc[len(df)] = df1.ix[:1]
    
    df = df.set_index(series2)
    
    return df

In [542]:
def getStats(sensor, eventTuples):

    '''
    NAME
        getStats
    
    DESCRIPTION
        Performs statistical analysis on data from one sensor and one event
        
    INPUT
        sensor:       sensor to analyze data from
        eventTuples:  run, database info
    
    RETURNS
        Prints mean, max, min, standard deviation, skew, and kurtosis
    
    EXAMPLE
        sensor = 'rct_rtd1'
        
        getStats(sensor, eventTuples)
    '''
    
    data = getEngData(sensor, eventTuples)
    mean = sp.stats.tmean(data)
    maxval = sp.stats.tmax(data)
    minval = sp.stats.tmin(data)
    std = sp.stats.tstd(data)   
    skew = sp.stats.skew(data)
    kurtosis = sp.stats.kurtosis(data, fisher=False)
    
    print "mean\t\t", mean, "\nmax\t\t", maxval, "\nmin\t\t", minval, "\ndeviation\t", std, "\nskew\t\t", skew, "\nkurtosis\t", kurtosis
    
    return

In [543]:
DF1 = getDFStats('amb_rtd1', eventTuples[10])
DF2 = getStats('amb_rtd1', eventTuples[10])

mean		22.6897675648 
max		[ 22.801] 
min		[ 22.586] 
deviation	0.0394312093093 
skew		[ 0.20623031] 
kurtosis	[ 2.29516189]


In [544]:
DF1

,amb_rtd1
count,3971.000000
mean,22.689768
std,0.039431
min,22.586000
25%,22.663000
50%,22.678000
75%,22.724000
max,22.801000
kurtosis,2.295162
skew,0.206308


In [532]:
kurtosis = sp.stats.kurtosis(data, fisher=False)

In [535]:
type(kurtosis)
kurtosis

array([ 1.5,  1.5])

In [536]:
DF1.loc[len(DF1)] = kurtosis[1]

In [537]:
DF1

,amb_rtd1
count,3971.000000
mean,22.689768
std,0.039431
min,22.586000
25%,22.663000
50%,22.678000
75%,22.724000
max,22.801000
kurtosis,-0.704346
skew,0.206308
